# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
!pip install pandas scikit-learn catboost
!pip install optuna

import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, RandomizedSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from google.colab import files
from scipy.stats import uniform, randint
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

train = pd.read_csv("./train_final (1).csv")

#validation set 분리

In [50]:

X = train.drop(columns=["y"])
y = train["y"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42)

#cross validation 세팅
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#model

In [51]:

models = {
    #"AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    #"GradientBoost": GradientBoostingClassifier(n_estimators=100, random_state=42),
    #"XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "CatBoost" : CatBoostClassifier(verbose=0, random_state=42),
    #"SVM" : SVC(kernel='rbf', probability=True, random_state=42)
}

# Evaluating

In [52]:
import numpy as np

X = train.drop(columns=["y"])
y = train["y"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2,  random_state=42)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {}
cv_results = {} #cross validation

scoring = ["accuracy", "f1", "roc_auc"]

for name, model in models.items():
    model.fit(X_train, y_train)
    #cross validation
    scores = cross_validate(
        model,
        X_train, y_train,
        cv=cv,
        scoring=scoring,
        return_train_score=False
    )
    cv_results[name] = {
        metric: scores[f"test_{metric}"].mean()
        for metric in scoring
    }

results_cv = pd.DataFrame(cv_results).T
results_cv["MeanScore"] = results_cv[["accuracy", "f1", "roc_auc"]].mean(axis=1)
print("Cross-Validation Results (5-fold):")
print(results_cv)


Cross-Validation Results (5-fold):
          accuracy        f1  roc_auc  MeanScore
CatBoost  0.643893  0.643675  0.69427   0.660612


In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble   import AdaBoostClassifier, GradientBoostingClassifier
from xgboost            import XGBClassifier
from scipy.stats        import uniform, randint
from catboost import CatBoostClassifier
from sklearn.svm import SVC
import numpy as np
import optuna

X = train.drop(columns=["y"])
y = train["y"]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2,  random_state=42)

#cross validation 세팅
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

models = {

    "GradientBoost": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "CatBoost" : CatBoostClassifier(verbose=0, random_state=42),

}

'''
#"AdaBoost": AdaBoostClassifier(n_estimators=100, random_state=42),
    "AdaBoost": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        # base_estimator 변화도 시도해볼 수 있음: DecisionTreeClassifier(max_depth=...)
        "SVM" : SVC(kernel='rbf', probability=True, random_state=42)
    },
    "SVM": {
        "C": uniform(0.1, 10),
        "kernel": ["rbf", "poly", "sigmoid"],
        "gamma": ["scale", "auto", "sigmoid"],
        "degree": randint(2, 5)
    }
    '''

param_dists = {

    "GradientBoost": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth":      randint(2, 8),
        "subsample":      uniform(0.6, 0.4)
    },
    "XGBoost": {
        "n_estimators":    randint(50, 300),
        "learning_rate":   uniform(0.01, 0.5),
        "max_depth":       randint(2, 8),
        "colsample_bytree": uniform(0.5, 0.5),
        "subsample":       uniform(0.6, 0.4)
    },
    "CatBoost": {
        "iterations": randint(100, 1000),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "l2_leaf_reg": uniform(1,10),
        "border_count": randint(32, 255),
        "bagging_temperature": uniform(0,1)
    },
}

def score(estimator, X_train, y_train):
  y_pred = estimator.predict(X_train)
  try:
    y_proba = estimator.predict_proba(X_train)[:, 1]
  except AttributeError:
    y_proba = estimator.decision_function(X_train)
  acc = accuracy_score(y_train, y_pred)
  f1 = f1_score(y_train, y_pred)
  auc = roc_auc_score(y_train, y_proba)
  return np.mean([acc, f1, auc])

scoring = score

# 4. RandomizedSearchCV 로 튜닝
best_estimators = {}
for name, model in models.items():
    print(f"=== Tuning {name} ===")
    search = RandomizedSearchCV(
        estimator       = model,
        param_distributions = param_dists[name],
        n_iter          = 30,
        scoring         = scoring,
        cv              = cv,
        random_state    = 42,
        n_jobs          = -1,
        verbose         = 1,
        refit           = True
    )
    search.fit(X_train, y_train)
    print(f"Best {name} params: {search.best_params_}")
    print(f"Best {name} CV score: {search.best_score_:.4f}\n")
    best_estimators[name] = search.best_estimator_


for name, model in best_estimators.items():
    model.fit(X_train, y_train)
    y_pred  = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:,1]
    print(f"--- {name} (Tuned) ---")
    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    auc = roc_auc_score(y_val, y_proba)
    print("Accuracy :", acc)
    print("F1-score :", f1)
    print("AUC      :", auc)
    print("Meanscore:", (acc+f1+auc)/3)

=== Tuning GradientBoost ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from scipy.stats import randint, uniform
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


# 1) 피처/타깃 분리 및 학습/검증셋 생성
X = train.drop(columns=["id", "y", "shares"], errors="ignore")
y = train["y"]
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 2) CV 객체
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 3) 모델 & 파라미터 분포 정의
models = {
    "GradientBoost": GradientBoostingClassifier(random_state=42),
    "XGBoost":       XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42),
    "CatBoost":      CatBoostClassifier(verbose=0, random_state=42),
}

param_dists = {
    "GradientBoost": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "subsample": uniform(0.6, 0.4),
    },
    "XGBoost": {
        "n_estimators": randint(50, 300),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "colsample_bytree": uniform(0.5, 0.5),
        "subsample": uniform(0.6, 0.4),
    },
    "CatBoost": {
        "iterations": randint(100, 1000),
        "learning_rate": uniform(0.01, 0.5),
        "max_depth": randint(2, 8),
        "l2_leaf_reg": uniform(1, 10),
        "border_count": randint(32, 255),
        "bagging_temperature": uniform(0, 1),
    },
}

# 4) 사용자 정의 스코어링 함수
def multi_score(estimator, X, y):
    y_pred = estimator.predict(X)
    # 확률 또는 decision function
    if hasattr(estimator, "predict_proba"):
        y_proba = estimator.predict_proba(X)[:, 1]
    else:
        y_proba = estimator.decision_function(X)
    acc = accuracy_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc = roc_auc_score(y, y_proba)
    return np.mean([acc, f1, auc])

# 5) RandomizedSearchCV 루프
best_estimators = {}
for name, model in models.items():
    print(f"=== Tuning {name} ===")
    search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_dists[name],
        n_iter=30,
        scoring=multi_score,
        cv=cv,
        random_state=42,
        n_jobs=-1,
        verbose=1,
        refit=True
    )
    search.fit(X_train, y_train)
    print(f"Best {name} params: {search.best_params_}")
    print(f"Best {name} CV score: {search.best_score_:.4f}\n")
    best_estimators[name] = search.best_estimator_

# 6) 튜닝된 모델 검증
for name, model in best_estimators.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    y_proba = model.predict_proba(X_val)[:, 1]
    print(f"--- {name} (Tuned) on Validation ---")
    print("Accuracy :", accuracy_score(y_val, y_pred))
    print("F1-score :", f1_score(y_val, y_pred))
    print("AUC      :", roc_auc_score(y_val, y_proba))
    print()


=== Tuning GradientBoost ===
Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 